In [2]:
import numpy as np
import pandas as pd
import cv2
import os
from tqdm import tqdm
import pytesseract
from glob import glob
import spacy
import re
import string

In [3]:
txt = """Google has released it's new model which has got attention of everyone.
Amazon is planning to expand into Food delivery, thereby giving competition .
Apple is coming up with new iphone model. Flipkart will have to catch up soon."""

In [4]:
whitespace = string.whitespace
punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~ÛÏ"
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
print(type(whitespace))
print(type(punctuation))
print(type(tablePunctuation))
print(type(tableWhitespace))
text = str(txt)
text = text.lower()
removewhitespace = text.translate(tableWhitespace)
removepunctuation = removewhitespace.translate(tablePunctuation)

print(str(removepunctuation))

<class 'str'>
<class 'str'>
<class 'dict'>
<class 'dict'>
googlehasreleaseditsnewmodelwhichhasgotattentionofeveryone.amazonisplanningtoexpandintofooddelivery,therebygivingcompetition.appleiscomingupwithnewiphonemodel.flipkartwillhavetocatchupsoon.


In [5]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~ÛÏ"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [6]:
import warnings 
warnings.filterwarnings('ignore')

In [7]:
model_ner = spacy.load('./output/model-best/')
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'

In [8]:
image = cv2.imread('./data/242.jpeg')

'''
cv2.imshow('businesscard', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
'''

testData = pytesseract.image_to_data(image)

testList = list(map(lambda x:x.split('\t'), testData.split('\n')))
df = pd.DataFrame(testList[1:], columns = testList[0])
df.dropna(inplace=True)
df['text'] = df['text'].apply(cleanText)

df_clean = df.query('text !=""')
content = " ".join([w for w in df_clean['text']])
print(content)
doc = model_ner(content)

dr. t. s. reddy senior consultant traffic transportation lea associates south asia pvt. ltd. leadership in consulting engineers planners engineering hl no 1-4-879/54/1a, street no. 8, p lannin g near vijaya bank, lower tank bund, gandhinagar, hyderabad-500080 a.p. india solutions tel 91-40-66747 122 / 66747135 . mobile 91-91778 82230 email tsr@lasaindia.com ————— limmappagari@ymait.com _ alea group company website - www.lasaindia.com


In [9]:
print(doc.ents)
for entity in doc.ents:
    print(entity.text,'--',entity.label_)

(dr., t., s., reddy, street, no., lannin, g, near, vijaya, bank, ,, lower, tank, bund, a.p., india, 91-40-66747, 66747135, 91-91778, 82230, tsr@lasaindia.com, —, —, —, —, —, limmappagari@ymait.com)
dr. -- B-NAME
t. -- I-NAME
s. -- I-NAME
reddy -- I-NAME
street -- B-ADD
no. -- I-ADD
lannin -- I-ADD
g -- I-ADD
near -- I-ADD
vijaya -- I-ADD
bank -- I-ADD
, -- I-ADD
lower -- I-ADD
tank -- I-ADD
bund -- I-ADD
a.p. -- I-ADD
india -- I-ADD
91-40-66747 -- B-PHONE
66747135 -- I-PHONE
91-91778 -- B-PHONE
82230 -- I-PHONE
tsr@lasaindia.com -- B-EMAIL
— -- I-EMAIL
— -- B-PHONE
— -- I-EMAIL
— -- I-EMAIL
— -- I-EMAIL
limmappagari@ymait.com -- B-EMAIL


In [10]:
from spacy import displacy

In [11]:
#displacy.serve(doc,style='ent')

In [12]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [13]:
doc_text = docjson['text']

In [14]:
datafram_tokens = pd.DataFrame(docjson['tokens'])
datafram_tokens['token'] = datafram_tokens[['start','end']].apply(
    lambda x:doc_text[x[0]:x[1]] , axis = 1)
datafram_tokens.head(10)

,id,start,end,token
0,0,0,2,dr
1,1,2,3,.
2,2,4,6,t.
3,3,7,9,s.
4,4,10,15,reddy
5,5,16,22,senior
6,6,23,33,consultant
7,7,34,41,traffic
8,8,42,56,transportation
9,9,57,60,lea


In [15]:
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
datafram_tokens = pd.merge(datafram_tokens,right_table,how='left',on='start')

In [16]:
datafram_tokens.fillna('O',inplace=True)
datafram_tokens.head(10)

,id,start,end,token,label
0,0,0,2,dr,B-NAME
1,1,2,3,.,O
2,2,4,6,t.,I-NAME
3,3,7,9,s.,I-NAME
4,4,10,15,reddy,I-NAME
5,5,16,22,senior,O
6,6,23,33,consultant,O
7,7,34,41,traffic,O
8,8,42,56,transportation,O
9,9,57,60,lea,O


In [17]:
def parser(text,label):
    if label == 'PHONE':
        text = text.lower()
        text = re.sub(r'\D','',text)
        
    elif label == 'EMAIL':
        text = text.lower()
        allow_special_char = '@_.\-'
        text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char),'',text)
        
    elif label == 'ADD':
        text = text.lower()
        allow_special_char = ':/.%#\-'
        text = re.sub(r'[^A-Za-z0-9{} ]'.format(allow_special_char),'',text)
        
    elif label in ('NAME', 'DES'):
        text = text.lower()
        text = re.sub(r'[^a-z ]','',text)
        text = text.title()
        
        
    return text

In [18]:
info_array = datafram_tokens[['token','label']].values
entities = dict(NAME=[],PHONE=[],EMAIL=[],ADD=[])
previous = 'O'

for token, label in info_array:
    bio_tag = label[0]
    label_tag = label[2:]
    
    text = parser(token,label_tag)
    
    if bio_tag in ('B','I'):
        
        if previous != label_tag:
            entities[label_tag].append(text)
            
        else:
            if bio_tag == "B":
                entities[label_tag].append(text)
                
            else:
                if label_tag in ("NAME", "ADD"):
                    entities[label_tag][-1] = entities[label_tag][-1] + " " + text
                    
                else:
                    entities[label_tag][-1] = entities[label_tag][-1] + text
                    
    
    
    previous = label_tag


In [19]:
s_name = ""
s_mail = ""
s_add = ""
s_phone = ""
entities.keys()
s_name = " ".join(entities['NAME'])
s_mail = " ".join(entities['EMAIL'])
s_add = " ".join(entities['ADD'])
s_phone = " ".join(entities['PHONE'])
print(s_name,"\n",s_phone,"\n",s_mail,"\n",s_add)

Dr T S Reddy 
 91 66747135 91 82230  
 tsr@lasaindia.com  limmappagari@ymait.com 
 street no lannin g near vijaya bank  lower tank bund a.p india


In [20]:
df = pd.read_csv("all_visiting_cards.csv")

In [21]:
d = {'id': 'first', 'text': 'sum'}
df_new = df.groupby('id', as_index=False).aggregate(d).reindex(columns=df.columns)

In [22]:
df_new.head()

,id,text
0,000.jpeg,"040-48528881,9030952549Fi/laurelsoverseaseduc..."
1,001.jpeg,johnsmithmarketingmanagerweb:www.psdgraphics.c...
2,002.jpeg,"a80984892444108672,224441/ENKATESWARA& ="
3,003.jpeg,"Prasad@9963173535,9490454585 FlexDesign#Al..."
4,004.jpeg,<=>MadisonYachtSalesPaulMadisonYachtBroker-Nor...


In [23]:

df.iloc[1,1]+df.iloc[2,1]

'040-48528881,'

In [24]:
img_cv = cv2.imread('./Selected/052.jpeg')

In [25]:
text_cv = pytesseract.image_to_string(img_cv)
text_cv

'HONDA\n\nDONNIE HANSEN\nMOTOCROSS TEAM RIDER\nTEAM HONDA\n\nAMERICAN HONDA MOTOR CO., INC.\n100 WEST ALONDRA BOULEVARD e GARDENA, CA 90247\n(213) 321-8680\n'

In [26]:
new_text = text_cv.replace("\n"," ")

In [27]:
new_text = new_text.lower()

In [61]:
imgPaths= glob('./Selected/*.jpeg')

In [55]:
new_all_visiting_cards = pd.DataFrame(columns=['id','text'])
for imgPath in tqdm(imgPaths,desc = 'VisitingCards'):
    l = []
    m = []
    directory, filename = os.path.split(imgPath)
    m.append(filename)
    image = cv2.imread(imgPath)
    data = pytesseract.image_to_string(image)
    new_data = data.replace("\n"," ")
    l.append(new_data)
    visiting_cards = pd.DataFrame(list(zip(m,l)),columns=['id','text'])
    new_all_visiting_cards = pd.concat((new_all_visiting_cards,visiting_cards))

VisitingCards: 100%|█████████████████████████████████████████████████████████████████| 293/293 [02:15<00:00,  2.17it/s]


In [56]:
new_all_visiting_cards.reset_index(drop=True,inplace=True)
new_all_visiting_cards

,id,text
0,000.jpeg,"‘. 040-4852 8881, 90309 52549 Fi /laurelsovers..."
1,001.jpeg,john smith marketing manager web: www.psdgra...
2,002.jpeg,"a8 0 98489 24441 lini dy 08672, 224441 1 /ENK..."
3,003.jpeg,"Prasad @ 9963173535,9490454585 i Flex Design ..."
4,004.jpeg,<=> Madison Yacht Sales Paul Madison Yacht Br...
...,...,...
288,288.jpeg,D Inside Design Ankit Dhyani ankit@insidedes...
289,289.jpeg,YOUR PRATICE COMPANY NAME A TAG LINE OF YOUR S...
290,290.jpeg,1) ne
291,291.jpeg,XXXXX XXXXXX G FIC Lyf ( Designation ) Mob. :...


In [81]:
def cleanText(xyz):
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(xyz)
    text = text.lower()
    removepunctuation = text.translate(tablePunctuation)
    
    return str(removepunctuation)+" "

In [82]:
dic = []
for imgPath in tqdm(imgPaths,desc = 'VisitingCards'):
    directory, filename = os.path.split(imgPath)
    #m.append(filename)
    
    
    image = cv2.imread(imgPath)
    data = pytesseract.image_to_string(image)
    #new_data = data.replace("\n"," ")
    dictionary ={"text":cleanText(data) ,"meta":{"source":filename}}
    dic.append(dictionary)
    
    #l.append(new_data)
    #visiting_cards = pd.DataFrame(list(zip(m,l)),columns=['id','text'])
    #new_all_visiting_cards = pd.concat((new_all_visiting_cards,visiting_cards))

VisitingCards: 100%|█████████████████████████████████████████████████████████████████| 293/293 [02:13<00:00,  2.19it/s]


In [83]:
import json
with open("i_am_so_happy.json", "w") as final:
    json.dump(dic, final)